In [12]:
# Import libraries and packages
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import requests
from fredapi import Fred
import panel as pn
pn.extension('plotly')
import plotly.express as px
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import holoviews as hv
from holoviews import opts

In [2]:
# Set API keys
load_dotenv("FRED_API_KEY.env")
fred_api_key = os.getenv("FRED_API_KEY")
# Check API data type
type(fred_api_key)

str

In [3]:
# Set API call for Monetary Supply*
fred = Fred(api_key=fred_api_key)
M2_data = fred.get_series('M2SL')
#Create M2 data frame
M2_data = pd.DataFrame(M2_data).reset_index()
#Define columns
columns = ['Date', 'Money Supply']
#Re-assign columns
M2_data.columns = columns

In [4]:
#Read csv data for S&P 500 from google finance
sp_500_path = Path("sp_500.csv")
SP_500_data = pd.read_csv(sp_500_path)
#Read csv data for DOW from google finance
dow_path = Path("dow.csv")
DOW_data = pd.read_csv(dow_path)
#Read csv data for NASDAQ from google finance
nasdaq_path = Path("nasdaq.csv")
NASDAQ_data = pd.read_csv(nasdaq_path)

In [5]:
#Clean data
##Drop nulls
M2_data = M2_data.dropna()
SP_500_data = SP_500_data.dropna()
DOW_data = DOW_data.dropna()
NASDAQ_data = NASDAQ_data.dropna()
#Remove time stamp from date column
SP_500_data['Date'] = SP_500_data['Date'].str.split(' ',expand=True)[0]
NASDAQ_data['Date'] = NASDAQ_data['Date'].str.split(' ',expand=True)[0]
DOW_data['Date'] = DOW_data['Date'].str.split(' ',expand=True)[0]
#Set index to date
M2_data.set_index(pd.to_datetime(M2_data['Date'], infer_datetime_format=True), inplace=True)
M2_data.drop(columns=["Date"], inplace=True)
SP_500_data.set_index(pd.to_datetime(SP_500_data['Date'], infer_datetime_format=True), inplace=True)
SP_500_data.drop(columns=["Date"], inplace=True)
DOW_data.set_index(pd.to_datetime(DOW_data['Date'], infer_datetime_format=True), inplace=True)
DOW_data.drop(columns=["Date"], inplace=True)
NASDAQ_data.set_index(pd.to_datetime(NASDAQ_data['Date'], infer_datetime_format=True), inplace=True)
NASDAQ_data.drop(columns=["Date"], inplace=True)

In [50]:
#Join data frames together
comparison_df = pd.concat([M2_data, SP_500_data, DOW_data, NASDAQ_data], axis = 'columns', join = 'inner')
#Define columns
columns = ['Money Supply', 'S&P 500', 'DOW', 'NASDAQ']
#Re-assign columns
comparison_df.columns = columns
#Write comparison to a csv
comparison_df.to_csv("Stock Index Comparison.csv", index = True, header = True)

In [52]:
#Plot comparison
comparison_plot = comparison_df.hvplot(title = "Money Supply v.s. Major Stock Indices", width = 1000, height = 500, shared_axes = False)
comparison_plot.opts(norm={'axiswise':True})

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [55]:
#Define time periods of interest to highlight on plot
pandemic_startLine = hv.VSpan(datetime.datetime(2020, 1, 1),datetime.datetime(2021, 7, 1)).options(color='red')
financialCrisis_startLine = hv.VSpan(datetime.datetime(2007, 6, 1),datetime.datetime(2009, 12, 31)).options(color='red')
september11_startLine = hv.VLine(datetime.datetime(2001, 9, 11)).options(color='red')

#Generate plot with time periods shown
formatted_comparison_df = comparison_df.hvplot(width=1000, height=500) * pandemic_startLine * hv.Text(datetime.datetime(2020, 1, 1), 21000, "Global Pandemic", halign='right') * \
financialCrisis_startLine * hv.Text(datetime.datetime(2007, 6, 1), 19500, "Financial Crisis", halign='center') * \
september11_startLine * hv.Text(datetime.datetime(2001, 9, 11), 18000, "Sept. 11 ", halign='right')
formatted_comparison_df

:Overlay
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .VSpan.I     :VSpan   [x,y]
   .Text.I      :Text   [x,y]
   .VSpan.II    :VSpan   [x,y]
   .Text.II     :Text   [x,y]
   .VLine.I     :VLine   [x,y]
   .Text.III    :Text   [x,y]

In [17]:
#Financial crisis
financial_crisis = comparison_df['2007-6-1':'2009-12-31']
financial_crisis.hvplot(title = "Money Supply v.s. Major Stock Indices During the Financial Crisis", width = 1000, height = 500)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [18]:
#9/11
nine_eleven = comparison_df['2001-1-1':'2001-12-31']
nine_eleven.hvplot(title = "Money Supply v.s. Major Stock Indices During 9/11", width = 1000, height = 500)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [19]:
#Pandemic
pandemic = comparison_df['2020-1-1':'2021-7-1']
pandemic.hvplot(title = "Money Supply v.s. Major Stock Indices During the Pandemic", width = 1000, height = 500)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [20]:
#Create correlation table for entire window
overall_correlation = comparison_df.corr()
overall_correlation

,Money Supply,S&P 500,DOW,NASDAQ
Money Supply,1.000000,0.954624,0.962897,0.934808
S&P 500,0.954624,1.000000,0.995841,0.971898
DOW,0.962897,0.995841,1.000000,0.957234
NASDAQ,0.934808,0.971898,0.957234,1.000000


In [56]:
#Create correlation graph for entire window
overall_correlation.hvplot.heatmap()

:HeatMap   [columns,index]   (value)

In [22]:
#Create correlation table for financial crisis
financial_crisis_correlation = financial_crisis.corr()
financial_crisis_correlation

,Money Supply,S&P 500,DOW,NASDAQ
Money Supply,1.000000,-0.923950,-0.922438,-0.833599
S&P 500,-0.923950,1.000000,0.996376,0.974627
DOW,-0.922438,0.996376,1.000000,0.963823
NASDAQ,-0.833599,0.974627,0.963823,1.000000


In [58]:
#Create correlation graph for finacial crisis
financial_crisis_correlation.hvplot.heatmap()

:HeatMap   [columns,index]   (value)

In [24]:
#Create correlation table for 9/11
nine_eleven_correlation = nine_eleven.corr()
nine_eleven_correlation

,Money Supply,S&P 500,DOW,NASDAQ
Money Supply,1.000000,-0.924462,-0.831640,-0.898802
S&P 500,-0.924462,1.000000,0.946032,0.972867
DOW,-0.831640,0.946032,1.000000,0.854483
NASDAQ,-0.898802,0.972867,0.854483,1.000000


In [59]:
#Create correlation graph for 9/11
nine_eleven_correlation.hvplot.heatmap()

:HeatMap   [columns,index]   (value)

In [26]:
#Create correlation table for pandemic
pandemic_correlation = pandemic.corr()
pandemic_correlation

,Money Supply,S&P 500,DOW,NASDAQ
Money Supply,1.000000,0.984860,0.984893,0.966648
S&P 500,0.984860,1.000000,0.996982,0.985310
DOW,0.984893,0.996982,1.000000,0.970028
NASDAQ,0.966648,0.985310,0.970028,1.000000


In [60]:
#Create correlation graph for pandemic
pandemic_correlation.hvplot.heatmap()

:HeatMap   [columns,index]   (value)

In [13]:
# References
## *https://github.com/mortada/fredapi